In [20]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tqdm
from tqdm import tqdm
import matplotlib.colors as mcolors
from copy import deepcopy
from gudhi.representations import Landscape, PersistenceImage

from topofisher.input_simulators.noisy_ring import CircleSimulator

from topofisher.filtrations.numpy.alphaDTML import AlphaDTMLayer

from topofisher.vectorizations.numpy.custom_vectorizations import TOPK
from topofisher.vectorizations.numpy.vectorization_layer import VectorizationLayers
from topofisher.vectorizations.numpy.shorthand_layers import PersistenceImages, PersistenceLandscapes

from topofisher.fisher.Fisher import show_fm_and_bias
from topofisher.fisher.imnn import IMNNLayer, MopedLayer, ExtraDimLayer
from topofisher.fisher.plot_fisher_stats import plotContours2D, plotSummaryDerivativeHists, plot_derivative_convergence


from topofisher.pipelines.circle import CirclePipeline
from topofisher.pipelines.convergence_checks import fisher_estimates_wrt_step_size
from topofisher.pipelines.utils import readFromFile, writeToFile
import tensorflow_probability as tfp
import topofisher

In [61]:
def test_run(theta_fid = tf.constant([1.]), K = 0., num_samples = 10):
    sims = []
    derivs = []
    with tf.autodiff.ForwardAccumulator(theta_fid , tf.constant([1.])) as acc:
        normal_distribution = tfp.distributions.Normal(loc = theta_fid, scale = 1. + K * theta_fid)
        samples = normal_distribution.sample(num_samples)
        op = samples
    return op, tf.reshape(acc.jvp(op), [1, num_samples, 1])

In [47]:
sims

<tf.Tensor: shape=(10, 1), dtype=float32, numpy=
array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)>

In [90]:
sims, derivs = test_run(theta_fid = tf.constant([10.]), K = 0.5, num_samples =  10)
fish = topofisher.fisher.Fisher.baseFisher(sims, derivs)
show_fm_and_bias(fish)
fish.C.numpy()[0, 0], fish.err.numpy()[0, 0]

log FI =  -2.55
FM =  [0.08]
Fractional bias =  [0.09]
Constraints =  [3.5776775]


(10.59775, 0.006944445893168449)

In [51]:
derivs

<tf.Tensor: shape=(1, 10, 1), dtype=float32, numpy=
array([[[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]]], dtype=float32)>